In [27]:
import numpy as np
import pandas as pd
import random

%load_ext autoreload
%autoreload 2
import Estimator as est
from Optimizer import OptVariance
import importlib
importlib.reload(est)

np.set_printoptions(precision=2)

d = 10
dist_type = "uni"
est_type = "sw"
N=100000
total_repeat = 5
print("distribution=",dist_type, "estimitor=",est_type, 
      "total sample=",N, "total repeat num=",total_repeat)
x_max = 1
x_step_size = x_max/d
x_grid = x_step_size*np.arange(d)
print(x_grid)

eps_grid = 0.5*np.arange(1,6)
print(eps_grid)

if dist_type=='exp':
    x_q = np.exp(-3*x_grid)
    x_q = x_q/np.sum(x_q)
elif dist_type=='uni':
    x_q = np.ones(d)/d
else:
    print('warning: invaild distribution type!')
print(x_q)

filename = "data/%s_q_%d.csv" %(dist_type,d)
x_data = {'x': x_grid, 'q':x_q}
pd.DataFrame(x_data).to_csv(filename)
print(filename, 'saved')

idx_original = random.choices(np.arange(d),x_q,k=N)
sample_original = x_grid[idx_original]

for i in range(len(eps_grid)):
    eps= eps_grid[i]
    print('eps=%.2f'%(eps))

    if est_type == 'sw':
        a_grid, M = est.SquareWave(eps,x_grid)
    elif est_type == 'grr':
        a_grid, M = est.GenRandResp(eps, x_grid)
    else:
        print('warning: invalid estimator type!')

    filename = 'data/%s_M_%.2f_%d.csv'%(est_type,eps,d)
    pd.DataFrame(M).to_csv(filename)
    filename = 'data/%s_a_%.2f_%d.csv'%(est_type,eps,d)
    pd.DataFrame(a_grid).to_csv(filename)

    rand_num = np.zeros((N,d))
    for j in range(d):
        rand_num[:,j] = random.choices(a_grid,M[:,j],k=N)
    x_q_noisy = np.zeros((1,d))
    for k in range(total_repeat):
        idx_noise = random.choices(range(N),k=N)
        idx_pair = list(zip(idx_noise,idx_original))
        sample_perturbed = rand_num[tuple(zip(*idx_pair))]
        elements,counts = np.unique(sample_perturbed,return_counts=True)
        x_q_est = est.EM(M,counts,eps)
        x_q_noisy +=x_q_est
    x_q_noisy = x_q_noisy/np.sum(x_q_noisy)
    print("x_q_noisy=",x_q_noisy)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
distribution= uni estimitor= sw total sample= 100000 total repeat num= 5
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9]
[0.5 1.  1.5 2.  2.5]
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
data/uni_q_10.csv saved
eps=0.50
x_q_noisy= [[0.1  0.1  0.1  0.1  0.11 0.1  0.1  0.1  0.09 0.11]]
eps=1.00
x_q_noisy= [[0.1  0.1  0.1  0.1  0.11 0.09 0.1  0.09 0.1  0.1 ]]
eps=1.50
x_q_noisy= [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]
eps=2.00
x_q_noisy= [[0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.09 0.1  0.1 ]]
eps=2.50
x_q_noisy= [[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]]


In [38]:
import numpy as np
import scipy as sp
from scipy.optimize import linprog
def OptVariance(eps,a_grid,x_q): # the optimization problem
    da = len(a_grid)
    dx = len(x_q)
    eEps = np.exp(eps)
    x_grid = a_grid # we may change this in later update version
    
    # the inequality constraints
    A_ub = np.zeros((da*(dx-1)*dx,da*dx)) # coefficient matrix
    counter=0
    for i in range(da):
        for j in range(dx):
            for k in range(dx):
                A = np.zeros((da,dx))
                if k!=j:
                    A[i,j] = 1
                    A[i,k]=-eEps
                    A = np.reshape(A,(1,dx*da))
                    A_ub[counter,:] = A
                    counter+=1
    b_ub = np.zeros((1,da*dx*(dx-1))) # limit vector

    # the equality constraints
    A_eq = np.zeros((dx+da,da*dx))
    for j in range(dx):
        temp = np.zeros((da,dx))
        temp[:,j] = np.ones(da)
        A_eq[j,:] = np.reshape(temp, (1,da*dx))
    for i in range(da):
        temp = np.zeros((da,dx))
        temp[i,:] = x_q
        A_eq[dx+i] = np.reshape(temp,(1,da*dx))
    print(x_q.shape)
    b_eq = np.concatenate((np.ones((1,dx)),x_q),axis=None)
    print(b_eq)


    # the obejetive funciton
    C = np.zeros((da,dx))
    for j in range(dx):
        C[:,j] = np.power(a_grid-x_grid[j],2)*x_q[j] # we want minimal conditional variance
    C = np.reshape(C,(1,da*dx))
    
    # limites
    bounds = list(zip(np.zeros(da*dx),np.ones(da*dx)))
    res = linprog(C, A_ub=A_ub, b_ub=b_ub, A_eq = A_eq, b_eq = b_eq, bounds=bounds)
    M = np.reshape(res.x,(da,dx))
    return M


OptVariance(2,x_grid,x_q)

(10,)
[1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1]
[[0.   0.   0.   0.01 0.02 0.03 0.04 0.05 0.06 0.08]
 [0.   0.   0.   0.   0.01 0.02 0.03 0.04 0.05 0.06]
 [0.   0.   0.   0.   0.   0.01 0.02 0.03 0.04 0.05]
 [0.01 0.   0.   0.   0.   0.   0.01 0.02 0.03 0.04]
 [0.02 0.01 0.   0.   0.   0.   0.   0.01 0.02 0.03]
 [0.03 0.02 0.01 0.   0.   0.   0.   0.   0.01 0.02]
 [0.04 0.03 0.02 0.01 0.   0.   0.   0.   0.   0.01]
 [0.05 0.04 0.03 0.02 0.01 0.   0.   0.   0.   0.  ]
 [0.06 0.05 0.04 0.03 0.02 0.01 0.   0.   0.   0.  ]
 [0.08 0.06 0.05 0.04 0.03 0.02 0.01 0.   0.   0.  ]]
[(0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1.0), (0.0, 1